# Title: NBA Games
## Author: Andy Orie
## Date: 5th July 2022

**Objectives:**
    1. Read in the csv files, set the index to a common attribute across all the dataset to allow for combining.
    2. Investigate and then Clean the data by removing redundant and un-necessary columns.
    3. Combine the files and save for use later.

In [ ]:
# Global Settings and Imported Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os

%matplotlib inline
%config InlineBackend.figure_format='retina'

from matplotlib import rcParams
# Specify the figure size in inches, for both X, and Y axes
rcParams['figure.figsize'] = 12,5

from matplotlib import style
from IPython.display import display

In [ ]:
# Import script 'chkdir.py' to check the working directory. "Y" if you are in the correct working directory, else "N".

import chkdir

In [ ]:
# Read in the csv files to panda dataframes

games = pd.read_csv('games.csv', index_col=['GAME_DATE_EST'], parse_dates=True)

teams = pd.read_csv('teams.csv', index_col=['TEAM_ID'])

players = pd.read_csv('players.csv', index_col=['TEAM_ID'])

ranking = pd.read_csv('ranking.csv', index_col=['TEAM_ID'])

# Games Dataset Processing

In [ ]:
# Let's investigate the data in the 'games' dataframe

print('The shape of the "games" dataframe is: ',games.shape)
print(100*'-')
games.head(5)

In [ ]:
# Let's investigate the games dataframe data types to see if any need to be changed.

print(games.dtypes)

In [ ]:
# It looks like the data in two of the columns are the same. Let's check.

print('There are {:0d} unique elements in the HOME_TEAM_ID column'.format(games['HOME_TEAM_ID'].nunique()))
print()
print('There are {:0d} unique elements in the TEAM_ID_home column'.format(games['TEAM_ID_home'].nunique()))
print()
print('Are the elements in the HOME_TEAM_ID and TEAM_ID_home columns the same?:', games['HOME_TEAM_ID'].equals(games['TEAM_ID_home']))

In [ ]:
#It appears the 'GAME_STATUS_TEXT' column has 'Final' as the only value. Let's check.

print(f'There are', games['GAME_STATUS_TEXT'].value_counts(), 'values under the GAME_STATUS_TEXT column')
print()
print('There are {:0d} unique values in the GAME_STATUS_TEXT column'.format(games['GAME_STATUS_TEXT'].nunique()))

In [ ]:
# It looks like the data in another two of the columns are the same. Let's check.

print('There are {:0d} unique elements in the VISITOR_TEAM_ID column'.format(games['VISITOR_TEAM_ID'].nunique()))
print()
print('There are {:0d} unique elements in the TEAM_ID_away column'.format(games['TEAM_ID_away'].nunique()))
print()
print('Are the elements in the VISITOR_TEAM_ID and TEAM_ID_away columns the same?:', games['VISITOR_TEAM_ID'].equals(games['TEAM_ID_away']))

In [ ]:
# Let us remove the reduntant columns of TEAM_ID_home, TEAM_ID_away and GAMES_STATUS_TEXT columns.

games1 = games.drop(['TEAM_ID_home', 'TEAM_ID_away','GAME_STATUS_TEXT'], axis=1)
print('The "games" dataframe previous shape was:', games.shape)
print()
print('Now, the "games1" dataframe has a new shape of:', games1.shape)

In [ ]:
# Check how many NaN values there are in the games1 dataframe

num_null = games1.isnull().sum().sum()
print('There are {:0d} NaN values inside the "games1" dataframe'.format(num_null))
print(100*'-')
print('The # of nulls in the columns are: \n',games1.isnull().sum())

In [ ]:
# There appears to be 99 records with NaN values. Let's remove these since they are much less than 10% of the data.

games2 = games1.dropna(axis=0, inplace=False)

print('The new shape of the dataframe after dropping NaN is:', games2.shape)

In [ ]:
# Let's check the name of the index

print(games2.index.names)
print(100*'-')

# Let's rename the index to Game_Date

games2.index.names = ['GAME_DATE']

games2.head(5)

In [ ]:
# What is the period of time considered in the "games2" dataframe?

print(sorted(list(games2.index.year.unique())))
print(100*'-')
print(f'There are {len(games2.index.year.unique())} seasons considered in the data')

In [ ]:
# Let's split the index time stamp into years, months and day columns.

games2['GM_YR'] = games2.index.year
games2['GM_MTH'] = games2.index.month
games2['GM_DAY'] = games2.index.day
games2.head(5)

In [ ]:
# Let's reorder the columns so our new date info columns (i.e. last 3 columns) are to the front.

cols = games2.columns.tolist()
cols = cols[-3:] + cols[:-3]
games2 = games2[cols]
games2.head(5)

# Teams Dataset Analysis

In [ ]:
# Let's investigate the data in the 'teams' dataframe

print('The shape of the "teams" dataframe is: ',teams.shape)
print(100*'-')
teams.head(5)

In [ ]:
# Let's investigate the teams dataframe data types.

print(teams.dtypes)

In [ ]:
# Let's take a closer look at the data under the 'LEAGUE_ID' column.

print('There is {:0d} unique value under the "LEAGUE_ID" column'.format(teams['LEAGUE_ID'].nunique()))
print()
print('There a total of', teams['LEAGUE_ID'].value_counts().sum(), 'counts inside the "LEAGUE_ID" column')

In [ ]:
# Let's keep only a few columns for our initial assessment since I don't need the other features at this time.

teams = teams[['ABBREVIATION', 'NICKNAME', 'CITY', 'ARENACAPACITY', 'HEADCOACH']]
teams.head(5)

In [ ]:
teams.info()

In [ ]:
# How many teams are there? and what are their names?

print('There are {:0d} teams in the dataset'. format(teams['NICKNAME'].nunique()))
print(100*'-')
print(teams['NICKNAME'].unique())

In [ ]:
# Check for NaN values in the "teams" dataframe.

nan_teams = teams.isnull().sum().sum()
print('There are {:0d} NaN values in the "teams" dataframe'.format(nan_teams))

In [ ]:
# Identify the columns with NaN values.

print(teams.isnull().sum())

In [ ]:
# Identify the rows with NaN values under the ARENACAPACITY column.

teams[teams['ARENACAPACITY'].isna()]

In [ ]:
# Are there any Zero values in our data?

teams.isin([0]).any()

In [ ]:
# What column has the zero values?

teams.isin([0]).sum()

In [ ]:
# Which team has zero Arena Capacity?

teams[teams['ARENACAPACITY'] == 0]

In [ ]:
#Replace the zero Arena Capacity for Magic with the mean.

teams['ARENACAPACITY'] = teams['ARENACAPACITY'].replace(to_replace=0, value=teams['ARENACAPACITY'].mean(), inplace=False)

In [ ]:
# Let's fill all NaN with the mean of the Area Capacity.

teams.fillna(teams.mean(), inplace=True)

# Let's verify the changes were made.

teams.groupby(['NICKNAME']).describe()

In [ ]:
# Let's make a copy of the 'teams' dataframe so we can combine it with the 'players' dataframe.
comb_df = teams.copy()

print('The "comb_df" has a shape of:', comb_df.shape)

# Players Dataset Analysis

In [ ]:
# Let's investigate the data in the 'players' dataframe

print('The "players" dataframe has a shape of:', players.shape)
print(100*'-')
players.head(5)

In [ ]:
# How many Nulls/ NaN values are there in the dataset?

players.isna().sum().sum()

In [ ]:
# Are there any Zero values in our dataset?

players.isin([0]).any().any()

In [ ]:
# Let's combine the 'teams' and 'players' dataframes on the index using Approach1: 
# the join(how='outer') to include all rows.

comb_df = comb_df.join(players, how='outer')

print('The shape of the combined dataframes is:',comb_df.shape)

In [ ]:
# Let's combine the 'teams' and 'players' dataframes on the index using Approach2: the pd.merge() to include all rows.

#comb_df1 = pd.merge(teams,players,how='outer',left_index=True, right_index=True)

#print('The shape of the combined dataframes is:',comb_df1.shape)
#print()
#comb_df1.columns

In [ ]:
# Let's take a look at the joined dataset.

comb_df.head(5)

In [ ]:
# Let's investigate the games dataframe data types to see if any need to be changed.

comb_df.dtypes

In [ ]:
# How many and What Seasons are there in the comb_df dataset?

print('There are {:0d} seasons in the comb_df dataset'.format(comb_df['SEASON'].nunique()))
print()

period = sorted(list(comb_df['SEASON'].unique()))
period

In [ ]:
# Let's play around with the data for specific teams and years.
# Get specific data based on User input for the team nickname and the season.

team_listing = list(comb_df['NICKNAME'].unique())
period = list(comb_df['SEASON'].unique())

prompt = 'Please enter the team nickname you want to look at from the list shown, or "q" to quit? '
prompt_2 = 'What season would you like to look at between 2009 to 2019? '

while True:
    print('Here are the teams to choose from: \n', team_listing)
    print()
    team_name = input(prompt)
    print()
    if team_name=='q':
        break
    if team_name.title() not in team_listing:
        print('ALERT - Your team is not in the shown list. Please try again.')
        print()
        continue
    chk_season = int(input(prompt_2))
    if chk_season not in period:
        print('ALERT - Your year is not between 2009 and 2019. Please try again.')
        print()
        continue
    else:
        print(df1.get_group((team_name.title(), chk_season)))
        break

# Ranking Dataset Analysis

In [ ]:
# Let's investigate the "ranking" dataframe

print('The "ranking" dataframe has a shape of:', ranking.shape)
print()
print(ranking.dtypes)

In [ ]:
ranking.head(5)

In [ ]:
ranking.info()

In [ ]:
# Checking for NaN values

nan_ranking = ranking.isnull().sum().sum()
print('There are {:0d} NaN values inside the dataframe'.format(nan_ranking))

In [ ]:
# Let's see which columns have NaN values

rank_cols = ranking.columns
print(ranking[rank_cols].isna().sum())

In [ ]:
print('There is therefore', ranking.shape[0]-nan_ranking, 'non-NaN values under the RETURNTOPLAY column')

In [ ]:
#Let's see the number of unique values under the 'RETURNTOPLAY'.

print('There are -', ranking['RETURNTOPLAY'].value_counts(), '- values under the RETURNTOPLAY column')
print()
print('There are {:0d} unique values in the RETURNTOPLAY column'.format(ranking['RETURNTOPLAY'].nunique()))

In [ ]:
print('The unique SEASON_ID values are: ',ranking['SEASON_ID'].unique())
print()
print('There are {:1d} unique values in the SEASON_ID column'.format(ranking['SEASON_ID'].nunique()))

In [ ]:
# Let's create a copy of the ranking dataframe.

ranking_2 = ranking.copy()

In [ ]:
# Let's populate a new list called 'my_lst' with the year extracted from the SEASON_ID column.
my_ls = []

start_yr = 2002

for yr1 in ranking_2['SEASON_ID']:
    if int(str(yr1)[-4:]) == start_yr:
        my_ls.append(start_yr)
    elif int(str(yr1)[-4:]) == 2003:
        my_ls.append(2003)
    elif int(str(yr1)[-4:]) == 2004:
        my_ls.append(2004)
    elif int(str(yr1)[-4:]) == 2005:
        my_ls.append(2005)
    elif int(str(yr1)[-4:]) == 2006:
        my_ls.append(2006)
    elif int(str(yr1)[-4:]) == 2007:
        my_ls.append(2007)
    elif int(str(yr1)[-4:]) == 2008:
        my_ls.append(2008)
    elif int(str(yr1)[-4:]) == 2009:
        my_ls.append(2009)
    elif int(str(yr1)[-4:]) == 2010:
        my_ls.append(2010)
    elif int(str(yr1)[-4:]) == 2011:
        my_ls.append(2011)
    elif int(str(yr1)[-4:]) == 2012:
        my_ls.append(2012)
    elif int(str(yr1)[-4:]) == 2013:
        my_ls.append(2013)
    elif int(str(yr1)[-4:]) == 2014:
        my_ls.append(2014)
    elif int(str(yr1)[-4:]) == 2015:
        my_ls.append(2015)
    elif int(str(yr1)[-4:]) == 2016:
        my_ls.append(2016)
    elif int(str(yr1)[-4:]) == 2017:
        my_ls.append(2017)
    elif int(str(yr1)[-4:]) == 2018:
        my_ls.append(2018)
    elif int(str(yr1)[-4:]) == 2019:
        my_ls.append(2019)
    elif int(str(yr1)[-4:]) == 2020:
        my_ls.append(2020)
    elif int(str(yr1)[-4:]) == 2021:
        my_ls.append(2021)

In [ ]:
# Let's insert the years in 'my_ls' as a new column at index 4 in the ranking_2 dataframe.

ranking_2.insert(4, 'SEASON', my_ls)
ranking_2.head()

In [ ]:
print('There are', len(ranking_2['TEAM'].unique()), 'teams in our dataset.')

print()

print(list(ranking_2['TEAM'].unique()))

In [ ]:
# Let's only take a few columns

new_ranking = ranking_2[['CONFERENCE', 'TEAM', 'G', 'W', 'L', 'W_PCT', 'HOME_RECORD', 'ROAD_RECORD', 'SEASON']]
new_ranking

In [ ]:
# What period does the new_ranking dataset cover?

print('There are {:0d} seasons in the new_ranking dataset'.format(new_ranking['SEASON'].nunique()))
print()
print(sorted(list(new_ranking['SEASON'].unique())))

In [ ]:
# What period does the comb_df dataset cover?

print('There are {:0d} seasons in the new_ranking dataset'.format(comb_df['SEASON'].nunique()))
print()
print(sorted(list(comb_df['SEASON'].unique())))

In [ ]:
# Let's make the index into a column for easy merging

new_ranking.reset_index(inplace=True)
new_ranking.head()

In [ ]:
# Let's make the index into a column for easy merging in the comb_df.

comb_df.reset_index(inplace=True)
comb_df.head()

In [ ]:
# Let's combine the 'new_ranking' and 'comb_df' dataframes on the index using Approach2: the pd.merge() to include all rows.

comb_df2 = pd.merge(new_ranking,comb_df,how='outer',on = ['TEAM_ID', 'SEASON'])

In [ ]:
comb_df2.head(5)

In [ ]:
#Let's save the modified dataframe "games2" as "games_mod.csv" for later use.

games2.to_csv('games_mod.csv')

In [ ]:
# Let's save the combined dataframe "comb_df2" as "team_players_ranking.csv" for later use.

comb_df2.to_csv('team_players_ranking.csv')

*End of Code*